In [ ]:
#install xgboost, fasttext, datasets

#Import Libraries

from utils.model_lstm import LSTMClassifier
import torch
from utils.get_vectors import get_embedding_model
from utils.traditional_algorithms import get_traditional_models
from utils.print_results import print_models_probabilities
import os
from utils.networking import start_tcp_server
torch.manual_seed(0)

#HYPERPARAMETERS 
EMBEDDING_SIZE = 300   
SENTENCE_SIZE = 100
HIDDEN_SIZE = 150
NUM_LAYERS = 1
CHECKPOINT_DIR = 'checkpoint'
EMBEDDING_MODEL = "cc.en.300.bin"
DATA_DIR = 'data'
DATA_LEN = 205000
FILE_NAME = 'sentiment_data.csv'
IP = "127.0.0.1"
PORT = 10000

# Setting GPU as the default device for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Training traditional algorithms and loading LSTM model
embedding_model = get_embedding_model(os.path.join(DATA_DIR, EMBEDDING_MODEL))
LOGREG, RFClassifier, XGBClassifier, Vader = get_traditional_models(os.path.join(DATA_DIR, FILE_NAME), DATA_LEN, SENTENCE_SIZE, EMBEDDING_SIZE, embedding_model)
MODEL_LSTM = LSTMClassifier(output_size = 5, hidden_dim = HIDDEN_SIZE, num_layers = NUM_LAYERS, embedding_dim = EMBEDDING_SIZE, seq_len = SENTENCE_SIZE).to(device)
MODEL_LSTM.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, 'model_sentiment_'+str(NUM_LAYERS)+'.ckpt')))

#print_models_probabilities(embedding_model, LOGREG, RFClassifier, XGBClassifier, Vader, MODEL_LSTM, SENTENCE_SIZE, EMBEDDING_SIZE, device)
start_tcp_server(IP, PORT, embedding_model, LOGREG, RFClassifier, XGBClassifier, Vader, MODEL_LSTM, SENTENCE_SIZE, EMBEDDING_SIZE, device)